In [1]:
import re
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import string
import nltk
import warnings 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import train_test_split

C:\Users\HP\anaconda3\envs\Reacon\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\HP\anaconda3\envs\Reacon\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\HP\anaconda3\envs\Reacon\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
df = pd.read_csv("twitter.csv")

In [3]:
df = df.drop(["id"],axis = 1)

In [4]:
df.head()

,label,tweet
0,0,@user when a father is dysfunctional and is s...
1,0,@user @user thanks for #lyft credit i can't us...
2,0,bihday your majesty
3,0,#model i love u take with u all the time in ...
4,0,factsguide: society now #motivation


In [5]:
df.describe()

,label
count,31962.000000
mean,0.070146
std,0.255397
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31962 entries, 0 to 31961
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   31962 non-null  int64 
 1   tweet   31962 non-null  object
dtypes: int64(1), object(1)
memory usage: 499.5+ KB


In [7]:
df.isna().sum()

label    0
tweet    0
dtype: int64

In [8]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i,'',input_txt)
    return input_txt

In [9]:
df['tidy_tweet'] = np.vectorize(remove_pattern)(df['tweet'], '@[\w]*')

In [10]:
df['tidy_tweet'] = df['tidy_tweet'].str.replace('[^a-zA-Z#]+',' ')

C:\Users\HP\AppData\Local\Temp\ipykernel_8524\3055865967.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['tidy_tweet'] = df['tidy_tweet'].str.replace('[^a-zA-Z#]+',' ')


In [11]:
df['tidy_tweet'] = df['tidy_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w) > 3]))

In [12]:
tokenized_tweet = df['tidy_tweet'].apply(lambda x: x.split())

In [13]:
from nltk.stem.porter import *
stemmer = PorterStemmer()

tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x])

In [14]:
for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])


In [15]:
df['tidy_tweet']  = tokenized_tweet

In [16]:
df.drop(['tweet'],axis=1)

,label,tidy_tweet
0,0,when father dysfunct selfish drag kid into dys...
1,0,thank #lyft credit caus they offer wheelchair ...
2,0,bihday your majesti
3,0,#model love take with time
4,0,factsguid societi #motiv
...,...,...
31957,0,that youuu
31958,0,nina turner airwav tri wrap herself mantl genu...
31959,0,listen song monday morn work
31960,1,#sikh #templ vandalis #calgari #wso condemn


In [17]:
docs = list(df['tidy_tweet'])
tfidf_vectorizer = TfidfVectorizer(use_idf=True, max_features = 20000) 
tfidf_vectorizer_vectors = tfidf_vectorizer.fit_transform(docs)
docs = tfidf_vectorizer_vectors.toarray()

In [18]:
X = docs
Y = df['label']

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.7 , random_state=0)

In [20]:
X_train.shape

(22373, 20000)

In [21]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [22]:
model.score(X_test,y_test)
predictions = model.predict(X_test)

In [23]:
predictions

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)